In [1]:
import PyPDF2
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse

In [2]:
def get_doc_name(date):
    """Return the string pathname of the file."""
    return "p1-" + date.strftime('%B') + str(date.year) + ".pdf"

In [3]:
def extract_data_pdf(pdf_name):
    """Return list of strings (dates and prices mixed) from pdf document"""
    # Parse dates and prices for coffee from pdf
    pdfFileObj = open("data/" + pdf_name, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    pageObj = pdfReader.getPage(0)
    data = pageObj.extractText().split("\n")
    
    # Leave only date and prices
    start = data.index("Robustas")
    end = data.index("Average")
    return data[start + 1 : end]

In [6]:
def get_data_period(start_month=8, end_month=9, start_year=2017, end_year=2018):
    """
    Return list of lists
    
    Each sublist has it's first value a corresponding date, and rest of
    them are prices for different types of coffee
    
    Ex. ['01-Aug', '130.82', '158.74', '153.49', '136.18', '104.83']
    """
    # Get all dates in the period
    date_start = datetime.datetime(start_year, start_month, 1)
    date_end = datetime.datetime(end_year, end_month, 1)
    step = relativedelta(months=+1)
    all_months = []
    while date_start <= date_end:
        all_months.append(date_start)
        date_start += step
    pdf_names = [get_doc_name(date) for date in all_months]

    # Get all names of documents
    all_data = [extract_data_pdf(name) for name in pdf_names]
    
    # Flatten list
    all_data = [item for sublist in all_data for item in sublist]
    chunk_data = []
    
    # Find months and make new sublists
    temp = []
    for i in all_data:
        if not i.replace('.', '', 1).isdigit():
            chunk_data.append(temp)
            temp = []
        temp.append(i)
    
    return chunk_data[1:]

In [9]:
def get_data_frame(data):
    """Return dataframe with following columns:
    
    date - date of format DD-MM-YYYY
    composite_ind - ICO composite price indicator
    columbian_milds - price of columbian milds
    other_milds - price of all, but columbian milds
    brazilian_nat - price of brazilian naturals
    robustas - price of robusta
    """
    df = pd.DataFrame(data, 
                      columns=["date", "composite_ind", "columbian_milds", "other_milds", "brazilian_nat", "robustas"]
                     ).dropna().reset_index().drop(columns=["index"])
    
    # Change date format
    dates = [parse(i) for i in list(df.date)]
    for i in range(len(dates)):
        if i < 108:
            dates[i] = dates[i].replace(year=2017)
        dates[i] = dates[i].strftime('%Y-%m-%d')
    df["date"] = dates
    
    return df

In [13]:
df = get_data_frame(get_data_period())
df.head()

,date,composite_ind,columbian_milds,other_milds,brazilian_nat,robustas
0,2017-08-01,130.82,158.74,153.49,136.18,104.83
1,2017-08-02,133.23,162.06,156.95,139.18,105.86
2,2017-08-03,133.11,161.91,156.81,139.15,105.69
3,2017-08-04,133.05,161.81,156.85,138.97,105.67
4,2017-08-07,134.72,163.77,158.87,141.04,106.73


In [14]:
# Save as csv
pd.DataFrame.to_csv(df, "data.csv")